# Running microkinetic simulations with CARE

This notebook tutorial provides a basic example on how to run microkinetic modeling (MKM) simulations of CRNs built with CARE.
We provide two ways to run simulations: with solvers implemented with Scipy, or with Julia solvers. Both options employ BDF stiff solvers as default.

## Load CRN

Generate the CRN with the care_run script. We will use a small CRN as case study, the C1O2 (*ncc* = 1, *noc* = 2) in thermal conditions on Ru(10m11), defined in the ``src/care/scripts/example_c1o2.toml``

```bash
care_run -i path_to_example_c1o2.toml
```

In [ ]:
from pickle import load

with open('./C1O2_Ru10m11/crn.pkl', "rb") as pickle_file:
    crn = load(pickle_file)

print(crn)

Microkinetic simulations are performed by calling the class method ``ReactionNetwork.run_microkinetic()``

In [ ]:
print(crn.run_microkinetic.__doc__)

## MKM inputs

In [ ]:
T = 450  # Absolute temperature (K)
P = 1e6  # Total pressure (Pa)
operating_conditions = {'T': T, 'P': P}

y_CO = 0.3  # Mole fraction of CO in the feed
y_CO2 = 0.1  # Mole fraction of CO2 in the feed
y_H2 = 0.6  # Mole fraction of H2 in the feed
y_feed = {'CO': y_CO, 'CO2': y_CO2, 'H2': y_H2}

## MKM with Scipy



In [ ]:
results = crn.run_microkinetic(iv=y_feed, 
                               oc=operating_conditions, 
                               uq=False, 
                               thermo=False,
                               solver='Python',
                               barrier_threshold=3.0,   
                               ss_tol=1e-10,
                               tfin=1e7,
                               eapp=False, 
                               gpu=False)  


In [ ]:
print(results)

Observations:
- The output dictionary follows the same structure of a typical ODE run with SciPy, including additional entries related to the (i) steady-state reaction rates (``forward_rate``, ``backward_rate``, ``net_rate``), (ii) ``consumption rate`` (cr) matrix where cr_{i,j} corresponds to the consumption rate of species i due to reaction j (if negative the species is consumed, positive produced), (iii) intermediates labels ``inters``, (iv) ``run_graph`` in NetworkX format. ``y`` contains the final surface coverage for the species.
- On the author machine, Integration with SciPy took 0.74 seconds.

## MKM with Julia

### CPU

In [ ]:
results = crn.run_microkinetic(iv=y_feed, 
                               oc=operating_conditions, 
                               uq=False, 
                               thermo=False,
                               solver='Julia',
                               barrier_threshold=3.0,   
                               ss_tol=1e-10,
                               tfin=1e7,
                               eapp=False, 
                               gpu=False)  

Observations: 
- You will see that the Julia option takes more time than the SciPy-based option (23.6 seconds on the author machine vs. 0.8 seconds with SciPy), this is due to the initial overhead required to compile the required dependencies.

### GPU

You need a GPU with CUDA to run the next part of the code.

Fixing!

In [ ]:
results = crn.run_microkinetic(iv=y_feed, 
                               oc=operating_conditions, 
                               uq=False, 
                               thermo=False,
                               solver='Julia',
                               barrier_threshold=3.0,   
                               ss_tol=1e-10,
                               tfin=1e7,
                               eapp=False, 
                               gpu=True)  